## Generate subset of only one class of objects

In [ ]:
from open_anything_diffusion.metrics.trajectory import artflownet_loss, flow_metrics, normalize_trajectory

In [ ]:
import torch
def normalize_trajectory_new(pred):  # pred: bs * 1200, traj_len, 3
    pred = pred.reshape(-1, 1200, pred.shape[1], pred.shape[2])
    norm = torch.linalg.norm(pred, ord=2, dim=-1).float()
    norm = torch.max(norm, dim=1).values + 1e-6

    largest_mag: float = torch.linalg.norm(
        pred, ord=2, dim=-1
    ).max()
    print(norm)
    print(largest_mag)


    pred = pred / norm[:, None, :, None]
    return torch.flatten(pred, start_dim=0, end_dim=1)  # bs * 1200, traj_len, 3
    

In [ ]:
# Create a dataset.
from open_anything_diffusion.datasets.flow_trajectory import FlowTrajectoryDataModule
datamodule = FlowTrajectoryDataModule(
        root="/home/yishu/datasets/partnet-mobility",
        batch_size=64,
        num_workers=30,
        n_proc=2,
        seed=42,
        trajectory_len=5,  # Only used when training trajectory model
    )

In [ ]:
batch = next(iter(datamodule.train_dataloader())).cuda()

In [ ]:
batch.delta[batch.mask!=0]

In [ ]:
new_traj = normalize_trajectory_new(batch.delta)[batch.mask!=0]

In [ ]:
print(flow_metrics(batch.delta[batch.mask!=0], new_traj))